In [51]:
from sympy import *
from IPython.display import display, Latex

var('s R C T_s z_i')
tf = 1/(1+s*R*C)
bilinear = (2/T_s)*(1-z_i)/(1+z_i)
tf = tf.subs(s, bilinear).simplify()
pretty_print(tf)

import re
#Given a bilinearly transformed transfer function F, with z^-1 represented by var,
#Returns two sequences (A, B)
def bilinear_to_sequence(F, var):
    F_a = denom(F).as_poly(var).all_coeffs()[::-1]
    scalar = F_a[0]
    F_a = [(-x/scalar).simplify() for x in F_a[1:]]
    F_b = [(x/scalar).simplify() for x in numer(F).as_poly(var).all_coeffs()[::-1]]

    #reconstruct the function to double check...
    F_new = (sum(b_n*Pow(var, n) for (n, b_n) in enumerate(F_b)) /
        (1 - sum(a_n*Pow(var, (n+1)) for (n, a_n) in enumerate(F_a))))
    assert((F - F_new).simplify() == 0) #check your work!
    
    return (F_a, F_b)

def print_bilinear_sequence(A, B, input_tex, output_tex, overall_gain=1):
    def format_term(term, func, power):
        if bool(term == 0):
            return None
        if term != 1:
            r = latex(term)
            if denom(term) == 1:
                r = " ( " + r + " ) "
        else:
            r = ""
        r += " " + func
        if power != 0:
            r += " z^{{-{}}} ".format(power)
        return r
    terms_fmt = list(filter(None, map(lambda x: format_term(x[0]/overall_gain, input_tex, x[1]), zip(B, range(len(B))))))
    terms_fmt += filter(None, map(lambda x: format_term(x[0]/overall_gain, output_tex, x[1]+1), zip(A, range(len(A)))))
    terms_fmt = re.sub('\\+\\s*-', '-', " + ".join(terms_fmt))
    if overall_gain != 1:
        terms_fmt = latex(overall_gain) + "[" + terms_fmt + "]"
    display(Latex('$$' + output_tex + " = " + terms_fmt + '$$'))

 
(A, B) = bilinear_to_sequence(tf, z_i)
print_bilinear_sequence(A, B, "C(z)", "V(z)")

# RC = tau, T_R = n*tau
var('T_R tau K n')
tf = tf.subs(C, tau/R).simplify()
tf = tf.subs(tau, T_R/n).simplify()
# let K = 2T_R/(n*T_S)
tf = tf.subs(T_R, K*n*T_s/2)
(A, B) = bilinear_to_sequence(tf, z_i)
print_bilinear_sequence(A, B, "C(z)", "V(z)", 1/(K+1))
display(Latex("$$\\mathrm{Where} \\: K = " + latex(2*T_R/(n*T_s)) + " \\: \\mathrm{and} \\: T_R = " + latex(n*tau) + ' = n C R' + "$$"))


       -Tₛ⋅(zᵢ + 1)         
────────────────────────────
2⋅C⋅R⋅(zᵢ - 1) - Tₛ⋅(zᵢ + 1)


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:
#  Therefore, since (K-1)/(K+1) = (((K+1)-2)/(K+1) = 1- 2/(K+1)
#
#       V(z) = V(z)*z^-1 + (1 / (K + 1)) * (C(z) + C(z)*z^-1 - 2V(z)*z^-1)

In [47]:
from ipywidgets import interact, FloatSlider as slider
import matplotlib.pyplot as plt

def envelope(A, D, S, R, samp_rate, s_sample, num_samples):
    state = 0
    counter = 0
    X_prev = 0.0
    X = 1.0
    Y_prev = 0.0
    Y = 0.0
    K = A*samp_rate/4                # initialize for attack timing
    while counter < num_samples:
        X_prev = X
        if state == 0:               # attack phase
            if Y > 0.98:             #   hit the attack peak
                X = S                #   set CV
                K = D*samp_rate/4    #   set K for decay timing
                state = 1            #   state transition
        elif state == 1:             # decay/sustain phase
            if counter >= s_sample:  #   reached end of sustain
                X = 0                #   set CV
                K = R*samp_rate/4    #   set K for release timing
                state = 2
        tmp = Y
        Y = (X + X_prev + (K-1)*Y_prev)/(K+1)
        Y_prev = tmp
        yield Y
        counter += 1

@interact
def _(A=slider(min=0, max=2, step=0.1, value=0.3),
      D=slider(min=0, max=2, step=0.1, value=0.3),
      S=slider(min=0, max=1, step=0.1, value=0.5),
      R=slider(min=0, max=2, step=0.1, value=0.5)):
    samp_rate = 200
    plt.plot(
        [x/float(samp_rate) for x in range(samp_rate*6)],
        list(envelope(A, D, S, R, samp_rate, samp_rate*4, samp_rate*6))
    )
    plt.show()

interactive(children=(FloatSlider(value=0.3, description='A', max=2.0), FloatSlider(value=0.3, description='D'…